In [1]:
import os,sys,re
import pandas as pd
import numpy as np
import scipy as sp
from scipy import spatial
from tqdm import tqdm
import boost_histogram as bh
import pickle

In [2]:
ref_csv = pd.read_csv('/home/hrichter/IDLI_revisions/forZenodo/IDLI_Zenodo_SampleSheet.csv', sep=',')
os.chdir('/home/hrichter/IDLI_revisions/forZenodo/random_nucs/')

In [3]:
indices = np.array([0, 1])
ref_sub = ref_csv[ref_csv.INDEX.isin(indices)]

In [4]:
def eat_pickle(hmmFile):
    with open(hmmFile, 'rb') as fin:
        hmmdat = pickle.load(fin)
    return hmmdat

In [ ]:
#step 1: do IDLI! here for random nucs, up to 100,000 nucs per sample

label = "random_nucs" #we use this label when just taking random nucs, rather than those near a specific region

for idx, line in ref_sub.iterrows():
    label_out = line['SAMPLE']
    print(label_out)
    frames = {}
    
    pickles_1 = line['HMMRES_1']
    pickles_31 = line['HMMRES_31']
    pickles_51 = line['HMMRES_51']
    pickles_71 = line['HMMRES_71']
    pickles_101 = line['HMMRES_101']

    r0_acc = eat_pickle(line['HMMRES_1'])
    r1_acc = eat_pickle(line['HMMRES_31'])
    r2_acc = eat_pickle(line['HMMRES_51'])
    r3_acc = eat_pickle(line['HMMRES_71'])
    r4_acc = eat_pickle(line['HMMRES_101'])

    foot_1 = line['FOOTPRINT_1']
    foot_31 = line['FOOTPRINT_31']
    foot_51 = line['FOOTPRINT_51']
    foot_71 = line['FOOTPRINT_71']
    foot_101 = line['FOOTPRINT_101']
    
    frames['1'] = pd.read_table(foot_1 ,sep=',',header=0)
    frames['31'] = pd.read_table(foot_31 ,sep=',',header=0)
    frames['51'] = pd.read_table(foot_51 ,sep=',',header=0)
    frames['71'] = pd.read_table(foot_71 ,sep=',',header=0)
    frames['101'] = pd.read_table(foot_101 ,sep=',',header=0)
    
    dim = (160,180)
    vplots = {}
    for key in frames:
        vplots[key] = np.zeros(dim)
    
    ref_tot = frames['1']
    r1_tot = frames['31']
    r2_tot = frames['51']
    r3_tot = frames['71']
    r4_tot = frames['101']
    
    ref_frame = ref_tot[(ref_tot.length < 200) & (ref_tot.midpoint > 200)]
    r0_frame = ref_frame[(ref_frame.length < 80) & (ref_frame.length > 20)]
    r1_frame = r1_tot[(r1_tot.length < 200) & (r1_tot.length > 20)]
    r2_frame = r2_tot[(r2_tot.length < 200) & (r2_tot.length > 20)]
    r3_frame = r3_tot[(r3_tot.length < 200) & (r3_tot.length > 20)]
    r4_frame = r4_tot[(r4_tot.length < 200) & (r4_tot.length > 20)]    
    
    zmw_ids = []
    nuc_ids = []
    label_ids = []
    nuc_profiles_r0 = []
    nuc_profiles_r1 = []
    nuc_profiles_r2 = []
    nuc_profiles_r3 = []
    nuc_profiles_r4 = []

    r0_mat = []
    r1_mat = [] 
    r2_mat = []
    r3_mat = []
    r4_mat = []

    nuc_acc_r0 = []
    nuc_acc_r1 = []
    nuc_acc_r2 = []
    nuc_acc_r3 = []
    nuc_acc_r4 = []

    nuc_sizes = []
    nuc_count = 0

    zmws = list(r0_acc.keys())  # Extract keys from the r0_acc dictionary and convert to a list
        
    for i in tqdm(zmws):
        ref = int(i)

        #stops code at X number of nucleosomes - take how many you can computationally process, often 50-100k per sample
        #comment out if you want ALL nucs from a single sample (not recommended unless small number of reads)
        if nuc_count > 100000: 
            break
        
        nucs = ref_frame[(ref_frame.zmw == ref)]['midpoint'].values
        if len(nucs) == 0: continue
        if len(nucs) < 5: continue
        lengths = ref_frame[(ref_frame.zmw == ref)]['length'].values                 

        r0_sub = r0_frame[(r0_frame.zmw == ref)]
        r1_sub = r1_frame[(r1_frame.zmw == ref)]
        r2_sub = r2_frame[(r2_frame.zmw == ref)]
        r3_sub = r3_frame[(r3_frame.zmw == ref)]
        r4_sub = r4_frame[(r4_frame.zmw == ref)]

        dist_r0 = np.subtract.outer(r0_sub['midpoint'].values, nucs)
        dist_r0[~((dist_r0 < 100) & (dist_r0 > -100))] = np.nan
        mlen_r0 = np.subtract.outer(r0_sub['length'].values, np.zeros(len(nucs)))
        mlen_r0[~((dist_r0 < 100) & (dist_r0 > -100))] = np.nan    

        dist_r1 = np.subtract.outer(r1_sub['midpoint'].values, nucs)
        dist_r1[~((dist_r1 < 100) & (dist_r1 > -100))] = np.nan
        mlen_r1 = np.subtract.outer(r1_sub['length'].values, np.zeros(len(nucs)))
        mlen_r1[~((dist_r1 < 100) & (dist_r1 > -100))] = np.nan

        dist_r2 = np.subtract.outer(r2_sub['midpoint'].values, nucs)
        dist_r2[~((dist_r2 < 100) & (dist_r2 > -100))] = np.nan
        mlen_r2 = np.subtract.outer(r2_sub['length'].values, np.zeros(len(nucs)))
        mlen_r2[~((dist_r2 < 100) & (dist_r2 > -100))] = np.nan

        dist_r3 = np.subtract.outer(r3_sub['midpoint'].values, nucs)
        dist_r3[~((dist_r3 < 100) & (dist_r3 > -100))] = np.nan
        mlen_r3 = np.subtract.outer(r3_sub['length'].values, np.zeros(len(nucs)))
        mlen_r3[~((dist_r3 < 100) & (dist_r3 > -100))] = np.nan

        dist_r4 = np.subtract.outer(r4_sub['midpoint'].values, nucs)
        dist_r4[~((dist_r4 < 100) & (dist_r4 > -100))] = np.nan
        mlen_r4 = np.subtract.outer(r4_sub['length'].values, np.zeros(len(nucs)))
        mlen_r4[~((dist_r4 < 100) & (dist_r4 > -100))] = np.nan

        for j in range(len(nucs)):

            midpoint = nucs[j]
            size = lengths[j]
            coord1 = int(midpoint - 100)
            try:
                new_key = int(i)
                acc = r0_acc[new_key][coord1:coord1 + 200]
            except KeyError:
                prefix = list(r0_acc.keys())[0].split('/')[0]
                suffix = 'ccs'
                new_key = "%s/%s/%s" % (prefix,int(i),suffix)
                acc = r0_acc[new_key][coord1: coord1 + 200]
            if len(acc) < 200: continue
            zmw_r0 = acc
            zmw_r1 = r1_acc[new_key][coord1:coord1 + 200]
            zmw_r2 = r2_acc[new_key][coord1:coord1 + 200]
            zmw_r3 = r3_acc[new_key][coord1:coord1 + 200]
            zmw_r4 = r4_acc[new_key][coord1:coord1 + 200]

            r0_mat.append(zmw_r0)
            r1_mat.append(zmw_r1)
            r2_mat.append(zmw_r2)
            r3_mat.append(zmw_r3)
            r4_mat.append(zmw_r4)

            nuc_hist_r0 = bh.Histogram(bh.axis.Regular(40,20,200),bh.axis.Regular(40,-100,100))
            nuc_hist_r1 = bh.Histogram(bh.axis.Regular(40,20,200),bh.axis.Regular(40,-100,100))
            nuc_hist_r2 = bh.Histogram(bh.axis.Regular(40,20,200),bh.axis.Regular(40,-100,100))
            nuc_hist_r3 = bh.Histogram(bh.axis.Regular(40,20,200),bh.axis.Regular(40,-100,100))
            nuc_hist_r4 = bh.Histogram(bh.axis.Regular(40,20,200),bh.axis.Regular(40,-100,100))

            nuc_hist_r0.fill(mlen_r0[:,j],dist_r0[:,j])
            nuc_hist_r1.fill(mlen_r1[:,j],dist_r1[:,j])
            nuc_hist_r2.fill(mlen_r2[:,j],dist_r2[:,j])
            nuc_hist_r3.fill(mlen_r3[:,j],dist_r3[:,j])
            nuc_hist_r4.fill(mlen_r4[:,j],dist_r4[:,j])

            vals_r0 = nuc_hist_r0.values().flatten()
            vals_r1 = nuc_hist_r1.values().flatten()
            vals_r2 = nuc_hist_r2.values().flatten()
            vals_r3 = nuc_hist_r3.values().flatten()
            vals_r4 = nuc_hist_r4.values().flatten()

            nuc_profiles_r0.append(vals_r0)
            nuc_profiles_r1.append(vals_r1)
            nuc_profiles_r2.append(vals_r2)
            nuc_profiles_r3.append(vals_r3)
            nuc_profiles_r4.append(vals_r4)

            zmw_ids.append(i)
            nuc_ids.append(nucs[j])
            nuc_sizes.append(lengths[j])
            
            nuc_count += 1

    np.save('%s_%s_individual_nucs_r0.npy' % (label_out, label) , np.vstack(nuc_profiles_r0))
    np.save('%s_%s_individual_nucs_r1.npy' % (label_out, label) , np.vstack(nuc_profiles_r1))
    np.save('%s_%s_individual_nucs_r2.npy' % (label_out, label) , np.vstack(nuc_profiles_r2))
    np.save('%s_%s_individual_nucs_r3.npy' % (label_out, label) , np.vstack(nuc_profiles_r3))
    np.save('%s_%s_individual_nucs_r4.npy' % (label_out, label) , np.vstack(nuc_profiles_r4))

    np.save('%s_%s_acc_r0.npy' % (label_out, label), np.vstack(r0_mat))
    np.save('%s_%s_acc_r1.npy' % (label_out, label), np.vstack(r1_mat))
    np.save('%s_%s_acc_r2.npy' % (label_out, label), np.vstack(r2_mat))
    np.save('%s_%s_acc_r3.npy' % (label_out, label), np.vstack(r3_mat))
    np.save('%s_%s_acc_r4.npy' % (label_out, label), np.vstack(r4_mat))

    nuc_profiles_r0 = []
    nuc_profiles_r1 = []
    nuc_profiles_r2 = []
    nuc_profiles_r3 = []
    nuc_profiles_r4 = []

    fho = open('%s_%s_individual_labels.txt' % (label_out, label), 'w')

    for i in range(len(zmw_ids)):
        print("%s\t%s\t%s" % (zmw_ids[i], nuc_ids[i], nuc_sizes[i]), file = fho)

    fho.close()

E14_mESC_Rep37_Seq2_10k_sample


 25%|█████████████████████████████████████████▏                                                                                                                             | 2470/10000 [00:27<01:16, 98.85it/s]

In [ ]:
#Step 2: prep sampled nucleosomes for clustering

import os
import sys
import re
import pickle
import numpy as np
import pandas as pd
import scipy as sp
import scanpy as sc 
from tqdm import tqdm
from datetime import datetime 

# Set up paths
ref_csv = pd.read_csv('/home/hrichter/IDLI_revisions/forZenodo/IDLI_Zenodo_SampleSheet.csv', sep=',')
indices = np.array([0, 1])
ref_sub = ref_csv[ref_csv.INDEX.isin(indices)]

# Specify the number of nucleosomes to process per sample
nucleosomes_per_sample = 100000

# Storage lists for data & metadata
tot_try = [] 
tot_rplot = [] 
tot_maximum = [] 
tot_lens = [] 
metadata_list = []

print("Starting data loading and aggregation...")
for idx, line in tqdm(ref_sub.iterrows(), total=len(ref_sub)):
    sample = line['SAMPLE']
    peaks = "random_nucs"

    dir = '/home/hrichter/IDLI_revisions/forZenodo/random_nucs/'
    metadata_file = f"{dir}{sample}_{peaks}_individual_labels.txt"  

    try:
        # Load accessibility data (r1, r2, r3) - 140bp
        b_full = np.load(f"{dir}{sample}_{peaks}_acc_r1.npy")[:, 30:170]
        c_full = np.load(f"{dir}{sample}_{peaks}_acc_r2.npy")[:, 30:170]
        d_full = np.load(f"{dir}{sample}_{peaks}_acc_r3.npy")[:, 30:170]
        
        # Load individual nucleosome maximums
        maximum_r1 = np.load(f"{dir}{sample}_{peaks}_individual_nucs_r1.npy")
        maximum_r2 = np.load(f"{dir}{sample}_{peaks}_individual_nucs_r2.npy") 
        maximum_r3 = np.load(f"{dir}{sample}_{peaks}_individual_nucs_r3.npy") 
        maximum_summed = maximum_r1 + maximum_r2 + maximum_r3 

        # Load nucleosome lengths (for tot_lens)
        lengths = pd.read_csv(metadata_file, sep='\t', header=None)[2].values # Column 2 is length
        
        # Load metadata
        meta_df = pd.read_csv(metadata_file, sep="\t", header=None, names=["zmw", "midpoint", "length", "label"])
        
        # Ensure metadata aligns with data
        if len(meta_df) != b_full.shape[0]:
            print(f"Warning: Metadata length mismatch for {sample}. Skipping.")
            continue  

        # Limit the number of nucleosomes (for alignment)
        num_nucleosomes = min(nucleosomes_per_sample, b_full.shape[0])
        
        b_full = b_full[:num_nucleosomes]
        c_full = c_full[:num_nucleosomes]
        d_full = d_full[:num_nucleosomes]
        maximum_summed = maximum_summed[:num_nucleosomes] 
        lengths = lengths[:num_nucleosomes] 
        meta_df = meta_df.iloc[:num_nucleosomes]

        # Stack accessibility data for clustering (b[:, 10:-10], c[:, 10:-10], d[:, 10:-10])
        try_cluster = np.hstack([b_full[:, 10:-10], c_full[:, 10:-10], d_full[:, 10:-10]])

        # Stack accessibility data for rplot (b, c, d)
        try_rplot = np.hstack([b_full, c_full, d_full]) 

        # Store data & metadata
        tot_try.append(try_cluster)
        tot_rplot.append(try_rplot) 
        tot_maximum.append(maximum_summed) 
        tot_lens.append(lengths) 
        
        meta_df["sample"] = sample
        meta_df["peak_set"] = peaks
        metadata_list.append(meta_df)

    except FileNotFoundError:
        print(f"Missing files for {sample}, skipping.")
        continue

# Final matrices
matrix_cluster = np.vstack(tot_try)
matrix_rplot = np.vstack(tot_rplot)
matrix_maximum = np.vstack(tot_maximum)
array_lens = np.concatenate(tot_lens)

metadata_df = pd.concat(metadata_list, ignore_index=True)

# Define file_name for saving
base_file_name = "forZenodo_randomnucs"

# Define the working directory
wd = "/home/hrichter/IDLI_revisions/forZenodo/random_nucs/"
os.makedirs(wd, exist_ok=True) # Ensure directory exists

np.save(wd + f'{base_file_name}_rplot.npy', matrix_rplot)
np.save(wd + f'{base_file_name}_maximum.npy', matrix_maximum)
np.save(wd + f'{base_file_name}_tot_lens.npy', array_lens)
print("\nSuccessfully saved aligned plot data (rplot, maximum, tot_lens).")

# Convert to AnnData object, and save
mat = sc.AnnData(X=np.nan_to_num(matrix_cluster))
mat.obs = metadata_df
mat.write(wd + f"{base_file_name}_nucs.h5ad")

print("AnnData object successfully created with accessibility data and metadata for specified samples and nucleosomes.")


In [ ]:
#optional: plot summed horizon plot for all nucs processed

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib import cm

summed = np.sum(matrix_maximum,axis=0)
reshaped = summed.reshape(40,40)

imshow(sp.stats.zscore(reshaped[::-1], axis=1),cmap=cm.Spectral_r)
plt.show()

In [ ]:
#step 3: do the clustering, using the prior AnnData object

wd = "/home/hrichter/IDLI_revisions/forZenodo/random_nucs/"
base_file_name = "forZenodo_randomnucs"
res_str = '50' #what you want your clustering resolution to be saved as in the file name

mat = sc.read_h5ad(wd + f"{base_file_name}_nucs.h5ad")

# Function to cluster the data and store cluster metadata + parameters
def cluster_mats(mat, res=0.6, neighbors=15):
    """Clusters an AnnData object using Leiden clustering and adds cluster labels to metadata."""
    sc.tl.pca(mat)  
    sc.pp.neighbors(mat, metric='correlation', n_neighbors=neighbors)  
    sc.tl.leiden(mat, resolution=res)  

    mat.obs['cluster'] = mat.obs['leiden'].astype(int)  # Convert cluster labels to integers

    # Store clustering parameters in `uns`
    clustering_params = {
        "neighbors": neighbors,
        "resolution": res,
        "clustering_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    mat.uns["clustering"] = clustering_params

    return mat  

# Perform clustering and update metadata
mat = cluster_mats(mat, res=0.5, neighbors=15) #can change resolution/neighbors here
mat.write(wd + f"{base_file_name}_nucs.h5ad")

# Save cluster assignments separately for downstream file-making
clusters = mat.obs['cluster'].values
np.save(wd + f"try_cluster_res_{res_str}_{base_file_name}.npy", clusters)

print("Clustering complete. Unique clusters:", np.unique(clusters))
print("Stored clustering parameters:", mat.uns["clustering"])

In [ ]:
#step 4: makes files for plotting clusters in R to check cluster identities

import os,sys,re
import pandas as pd
import numpy as np
import scipy as sp
from scipy import spatial
import scipy.stats
from tqdm import tqdm
import scanpy as sc

# Setup
base_path = '/home/hrichter/IDLI_revisions/forZenodo/random_nucs/'
os.chdir(base_path)
base_file_name = "forZenodo_randomnucs"
res_str = '50' 

clusters = np.load(base_path + f'try_cluster_res_{res_str}_{base_file_name}.npy', allow_pickle = True)

try_rplot = np.load(base_path + f'{base_file_name}_rplot.npy', allow_pickle=True)
maximum = np.load(base_path + f'{base_file_name}_maximum.npy', allow_pickle=True)
tot_lens = np.load(base_path + f'{base_file_name}_tot_lens.npy', allow_pickle=True)

# Initial Data Check
print(f"Clusters array shape: {clusters.shape}")
print("-" * 30)

if not (len(clusters) == len(try_rplot) == len(maximum) == len(tot_lens)):
    print("FATAL ERROR: Aligned data array lengths do not match cluster array length. Check 'step 2' code.")
    sys.exit(1)

target = 0.99 * len(clusters) #can adjust depending on whethe
cusum = 0
unique_clusters = np.unique(clusters)
cluster_sizes = {str(c): len(clusters[clusters == c]) for c in unique_clusters} 
sorted_clusters = sorted([int(c) for c in unique_clusters]) 

fho = open(base_path + f'plotme_vplots_clusters_{base_file_name}_res{res_str}.txt', 'w')
fho2 = open(base_path + f'plotme_accplots_clusters_{base_file_name}_res{res_str}.txt', 'w')
fho3 = open(base_path + f'plotme_lengths_clusters_{base_file_name}_res{res_str}.txt', 'w')

for i in sorted_clusters:
    # i is the integer cluster ID (e.g., 0, 1, 2)
    cluster_label_str = str(i) #

    current_size = cluster_sizes.get(cluster_label_str, 0) 

    cusum += current_size
    
    print(f"\nProcessing cluster {i}")
    print(f"Cluster size: {current_size}")
    
    cluster_mask = (clusters == i)

    sub = maximum[cluster_mask]

    if sub.shape[0] == 0:
        print(f"ERROR: 'maximum' subset for cluster {i} is empty. Skipping cluster.")
        continue
        
    summed = np.sum(sub, axis=0)
    # reshaped = summed.reshape(40, 40) # Original size 40x40 = 1600
    try:
        reshaped = summed.reshape(40, 40)
    except ValueError as e:
        print(f"Error reshaping 'maximum' for cluster {i}: {e}. Skipping.")
        continue
        
    plotter = sp.stats.zscore(reshaped[::-1], axis=1) # Reverse rows and z-score across columns

    # Print V-plot data to fho
    for j in range(21, 200):
        for k in range(-100, 100):
            ci = int((200 - j) * 40 / 180)
            cj = int((k + 100) / 5)
            if 0 <= ci < 40 and 0 <= cj < 40:
                print("%s\t%s\t%s\t%s" % (j, k, plotter[ci][cj], i + 1), file=fho)

    sub_acc = try_rplot[cluster_mask]
    
    acc = sub_acc[:, :140]
    acc2 = sub_acc[:, 140:280]
    acc3 = sub_acc[:, 280:]
    
    acc_mean = np.nanmean(acc, axis=0)
    acc2_mean = np.nanmean(acc2, axis=0)
    acc3_mean = np.nanmean(acc3, axis=0)
    
    # Print accessibility data to fho2
    for j in range(140):
        x_coord = j - 70 
        y_coord1 = acc_mean[j] if not np.isnan(acc_mean[j]) else 0
        y_coord2 = acc2_mean[j] if not np.isnan(acc2_mean[j]) else 0
        y_coord3 = acc3_mean[j] if not np.isnan(acc3_mean[j]) else 0
        
        print("%s\t%s\t%s\t31" % (x_coord, y_coord1, i + 1), file=fho2)
        print("%s\t%s\t%s\t51" % (x_coord, y_coord2, i + 1), file=fho2)
        print("%s\t%s\t%s\t71" % (x_coord, y_coord3, i + 1), file=fho2)

    sublens = tot_lens[cluster_mask]
    
    # Print length data to fho3
    for j in range(len(sublens)):
        print("%s\t%s" % (sublens[j], i + 1), file=fho3)
        
    if cusum >= target:
        print(f"\nReached {cusum/len(clusters) * 100:.2f}% coverage. Stopping after cluster {i}.")
        break 

fho.close()
fho2.close()
fho3.close()

print("\nPlot data generation complete using newly aligned arrays.")